In [1]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# =========================
# Baseline (Alphabetical) route üretimi
# Girdi:
#  - phase4_assignments_fixed.csv  (date, vehicle_id, vehicle_type, Mahalle)
#  - address_data_eng.csv          (neighborhood, street_latitude, street_longitude)
# Çıktı:
#  - phase5_baseline_routes.csv
#  - phase5_baseline_vehicle_summary.csv
# =========================

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    return 2 * R * atan2(sqrt(a), sqrt(1 - a))

def route_distance(route):
    return sum(
        haversine(
            route[i-1]["lat"], route[i-1]["lon"],
            route[i]["lat"], route[i]["lon"]
        )
        for i in range(1, len(route))
    )

def norm(x):
    return str(x).strip().lower()

# 1) Oku
assignments = pd.read_csv("phase4_assignments_fixed.csv")
addresses   = pd.read_csv("address_data_eng.csv", low_memory=False)

# 2) Kolonlar
addr_neigh_col = "neighborhood"
lat_col = "street_latitude"
lon_col = "street_longitude"

# 3) Normalize
assignments["Mahalle_norm"] = assignments["Mahalle"].apply(norm)
addresses["Mahalle_norm"]   = addresses[addr_neigh_col].apply(norm)

# 4) lat/lon temizle
addresses[lat_col] = pd.to_numeric(addresses[lat_col], errors="coerce")
addresses[lon_col] = pd.to_numeric(addresses[lon_col], errors="coerce")
addresses = addresses.dropna(subset=[lat_col, lon_col])

# 5) Koordinat (median)
coords = (
    addresses
    .groupby("Mahalle_norm")[[lat_col, lon_col]]
    .median()
    .reset_index()
    .rename(columns={lat_col: "latitude", lon_col: "longitude"})
)

# 6) Merge
df = assignments.merge(coords, on="Mahalle_norm", how="left")

# 7) Baseline (Alphabetical) routes
routes = []
vehicle_summary = []

for (date, vehicle_id), group in df.groupby(["date", "vehicle_id"]):
    # Koordinatı olan satırlar
    g = group.dropna(subset=["latitude", "longitude"]).copy()
    if g.empty:
        continue

    # Baseline tanımı: Mahalle adına göre alfabetik sıra
    g["Mahalle_sort"] = g["Mahalle"].astype(str)
    g = g.sort_values("Mahalle_sort", kind="mergesort")

    points = [
        {"Mahalle": r["Mahalle"], "lat": r["latitude"], "lon": r["longitude"]}
        for _, r in g.iterrows()
    ]

    total_km = route_distance(points)
    prev = None

    for idx, stop in enumerate(points):
        dist_prev = 0 if prev is None else haversine(
            prev["lat"], prev["lon"], stop["lat"], stop["lon"]
        )
        routes.append({
            "date": date,
            "vehicle_id": vehicle_id,
            "vehicle_type": group["vehicle_type"].iloc[0],
            "stop_order": idx + 1,
            "Mahalle": stop["Mahalle"],
            "latitude": stop["lat"],
            "longitude": stop["lon"],
            "distance_from_prev_km": round(dist_prev, 3)
        })
        prev = stop

    vehicle_summary.append({
        "date": date,
        "vehicle_id": vehicle_id,
        "vehicle_type": group["vehicle_type"].iloc[0],
        "stops": len(points),
        "total_distance_km": round(total_km, 2)
    })

# 8) Kaydet
pd.DataFrame(routes).to_csv("phase5_baseline_routes.csv", index=False)
pd.DataFrame(vehicle_summary).to_csv("phase5_baseline_vehicle_summary.csv", index=False)

print("✅ Baseline routes tamamlandı.")
print("-> phase5_baseline_routes.csv")
print("-> phase5_baseline_vehicle_summary.csv")


✅ Baseline routes tamamlandı.
-> phase5_baseline_routes.csv
-> phase5_baseline_vehicle_summary.csv


In [3]:
df = pd.read_csv("phase5_baseline_routes.csv")
df.head(10)

,date,vehicle_id,vehicle_type,stop_order,Mahalle,latitude,longitude,distance_from_prev_km
0,2025-12-19,1409,Large Garbage Truck,1,Ucevler Mh.,40.199857,28.947401,0.000
1,2025-12-19,1409,Large Garbage Truck,2,Ucevler Mh.,40.199857,28.947401,0.000
2,2025-12-19,1434,Large Garbage Truck,1,Alaaddinbey Mh.,40.208647,28.923144,0.000
3,2025-12-19,1434,Large Garbage Truck,2,Kadriye Mh.,40.092710,28.868163,13.713
4,2025-12-19,1434,Large Garbage Truck,3,Ucpinar Mh.,40.102980,28.840133,2.644
5,2025-12-19,1488,Large Garbage Truck,1,Atlas Mh.,40.122211,28.886940,0.000
6,2025-12-19,1488,Large Garbage Truck,2,Badirga Mh.,40.246969,28.694422,21.445
7,2025-12-19,1488,Large Garbage Truck,3,Baskoy Mh.,40.212266,28.749599,6.069
8,2025-12-19,1488,Large Garbage Truck,4,Catalagil Mh.,40.217634,28.705598,3.784
9,2025-12-19,1488,Large Garbage Truck,5,Cayli Mh.,40.274568,28.745238,7.169
